In [194]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import unicodedata
import re

In [195]:
df = pd.read_csv("../sources/export_06_18_24.csv", encoding='latin-1')

In [196]:
rf = pd.read_csv("../sources/lml-reconciled.csv", encoding='latin-1')

In [197]:
df['Secondary Catalog Number'] = df['Secondary Catalog Number'].fillna('')

In [198]:
rf['Secondary Catalog Number'] = rf['Secondary Catalog Number'].fillna('')

In [199]:
def get_catalog_number(smp):
    pcn = str(smp['Catalog Number'])
    scn = smp.fillna('')['Secondary Catalog Number']

    return pcn + scn

In [200]:
df['cat'] = [get_catalog_number(df.iloc[i]) for i in range(len(df))]

In [201]:
rf['cat'] = [get_catalog_number(rf.iloc[i]) for i in range(len(rf))]

In [202]:
rf = rf[['cat','Manufacturer Equivalent IDs','Brand Equivalent IDs']]

In [203]:
rf = rf.drop_duplicates()

In [204]:
df = df.join(rf.set_index('cat'),'cat')

In [205]:
df.ProcessName = df.ProcessName.apply(lambda x:x.strip('\t'))

In [206]:
df.Link = df.Link.apply(lambda x: 'http://vocab.getty.edu/' + "/".join(x.strip("#").split("/")[-2:]))

In [207]:
df.columns = ['photoid','year','circa','catalog','catalog2','man','bran','storfor','xd','gd','cd','td','s','omit',
              'locbox','locbag','expos','backp','processname','processlink','processaata','cat','manid','branid']

In [208]:
df = df[['cat','man','bran','year','circa','xd','gd','cd','td','s','storfor','omit',
        'locbox','locbag','expos','backp','processname','processlink','processaata','cat','manid','branid']]

In [209]:
# Autotype: http://viaf.org/viaf/131563321
# Foton: http://www.wikidata.org/entity/Q115823390

In [210]:
df.manid.loc[df.man=='Autotype'] = 'http://viaf.org/viaf/131563321'
df.manid.loc[df.man=='Foton'] = 'http://www.wikidata.org/entity/Q115823390'

In [211]:
df.manid.loc[df.man=='Kodak London'] = 'http://viaf.org/viaf/150021959'

In [212]:
mans = df.man.unique()

In [213]:
def to_filename(name):

    if pd.isna(name):
        return None

    name = name.lower().strip()
    
    # Normalize Unicode characters to their closest ASCII representation
    normalized_name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
    # Remove problematic characters
    safe_name = re.sub(r"[\/\\\:\*\?\"\<\>\|\#\%\&\{\}\\\<\>\*\?\$\/\'\"\:\@\+\`|=!]", "", normalized_name)
    # Replace spaces (and any residual space-like characters) with underscores
    safe_name = re.sub(r"\s+", "_", safe_name)
    # Append the .json extension
    return safe_name

In [214]:
df['mansafe'] = df.man.apply(to_filename)

In [215]:
len(df.mansafe.unique())==len(df.man.unique())

True

In [217]:
brans = df.bran.unique()

In [218]:
df['bransafe'] = df.bran.apply(to_filename)

In [219]:
len(df.bransafe.unique())==len(df.bran.unique())

True

In [221]:
df.to_pickle('reconc.pkl')